In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functions import *
from jacobi import *

In [2]:
%load_ext autoreload
%autoreload 2

In [51]:
%%bash
mkdir -p ../build
cd ../build
cmake ../
cmake --build . -j4
cd -

-- HDF5 CXX compiler wrapper is unable to compile a minimal HDF5 program.
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_HDF5Writer.cpp
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_filehandler.cpp
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_parser.cpp
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_utilities.cpp
-- Configuring done (0.3s)
-- Generating done (0.0s)
-- Build files have been written to: /home/balazs/Desktop/University/Research/Practice/Laplace/build
[  8%] Built target gtest
[ 12%] Building CXX object src/CMakeFiles/laplace_lib.dir/parser.cpp.o
[ 16%] Building CXX object src/CMakeFiles/laplace_lib.dir/calculator.cpp.o
[ 25%] Built target gtest_main
[ 33%] Built target gmock
[ 37%] Building CXX object src/CMakeFiles/laplace_lib.dir/utilities.cpp.o
[ 41%] Building CXX object src/CMakeFiles/laplace_lib.dir/HDF5Writer.cpp.o
[ 

In [52]:
%%bash
cd ../build/
ctest --output-on-failure
cd -

Test project /home/balazs/Desktop/University/Research/Practice/Laplace/build
      Start  1: HDF5WriterTest.WritesParametersCorrectly
 1/19 Test  #1: HDF5WriterTest.WritesParametersCorrectly .......   Passed    0.01 sec
      Start  2: HDF5WriterTest.WritesResultDatasetCorrectly


 2/19 Test  #2: HDF5WriterTest.WritesResultDatasetCorrectly ....   Passed    0.01 sec
      Start  3: HDF5WriterTest.CreatesExpectedFileStructure
 3/19 Test  #3: HDF5WriterTest.CreatesExpectedFileStructure ....   Passed    0.01 sec
      Start  4: HDF5WriterTest.OverwritesFileOnCreation
 4/19 Test  #4: HDF5WriterTest.OverwritesFileOnCreation ........   Passed    0.01 sec
      Start  5: FileHandlerTest.ValidReadAndWrite
 5/19 Test  #5: FileHandlerTest.ValidReadAndWrite ..............   Passed    0.00 sec
      Start  6: FileHandlerTest.InvalidFileReadThrows
 6/19 Test  #6: FileHandlerTest.InvalidFileReadThrows ..........   Passed    0.00 sec
      Start  7: FileHandlerTest.InconsistentRowLengthsThrows
 7/19 Test  #7: FileHandlerTest.InconsistentRowLengthsThrows ...   Passed    0.00 sec
      Start  8: FileHandlerTest.NonSquareMatrixThrows
 8/19 Test  #8: FileHandlerTest.NonSquareMatrixThrows ..........   Passed    0.00 sec
      Start  9: FileHandlerTest.InvalidNumberThrows
 9/19 Test 

In [3]:
generate_ex1(10, "Data")

In [53]:
!time ../build/src/laplace --config ../examples/config.txt --result-path ../examples/Data/serial_res_N10

Laplace equation solver
The used parameters are:
	N: 10
	tol: 0.001
	max iter: 500
	initial state path: ../examples/Data/data_N10.csv
	mask path: ../examples/Data/mask_N10.csv
	result path: ../examples/Data/serial_res_N10
	output: 2
	config path: ../examples/config.txt
Calculation finished after 51 iterations, the result is:
1, 0, 0, 0, 0, 0, 0, 0, 0, 0
1, 0.484444, 0.272562, 0.167656, 0.107517, 0.0695193, 0.0439469, 0.0257791, 0.0118973, 0
1, 0.665667, 0.438992, 0.291676, 0.194166, 0.12788, 0.081592, 0.0480866, 0.022241, 0
1, 0.74008, 0.527651, 0.368015, 0.251992, 0.168617, 0.10853, 0.0642627, 0.0297909, 0
1, 0.768146, 0.565656, 0.403605, 0.280398, 0.189272, 0.122443, 0.0727046, 0.0337513, 0
1, 0.768146, 0.565656, 0.403605, 0.280398, 0.189272, 0.122443, 0.0727046, 0.0337513, 0
1, 0.74008, 0.527651, 0.368015, 0.251992, 0.168617, 0.10853, 0.0642627, 0.0297909, 0
1, 0.665667, 0.438992, 0.291676, 0.194166, 0.12788, 0.081592, 0.0480866, 0.022241, 0
1, 0.484444, 0.272562, 0.167656, 0.107517

In [29]:
!time python jacobi.py --config config.txt --result-path ../examples/Data/python_res_N10.csv

Laplace equation solver
The used parameters are:
	N: 10
	tol: 0.001
	max iter: 500
	initial state path: ../examples/Data/data_N10.csv
	result path: ../examples/Data/python_res_N10.csv
	mask path: ../examples/Data/mask_N10.csv
	output: 2
	config path: config.txt
Calulation finished after 51 iterations, the result is:
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [1.         0.48444444 0.2725621  0.16765583 0.10751662 0.06951925
  0.04394686 0.02577907 0.01189731 0.        ]
 [1.         0.66566732 0.43899217 0.2916763  0.19416625 0.12787963
  0.08159195 0.04808656 0.02224099 0.        ]
 [1.         0.74008034 0.52765147 0.3680149  0.2519918  0.16861724
  0.10853033 0.06426273 0.02979093 0.        ]
 [1.         0.76814581 0.56565643 0.40360458 0.28039804 0.18927195
  0.12244285 0.07270464 0.0337513  0.        ]
 [1.         0.76814581 0.56565643 0.40360458 0.28039804 0.18927195
  0.12244285 0.07270464 0.0337513  0.        ]
